In [1]:
import json, csv, os, ast
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import scipy as sp
from IPython.display import display, HTML
import xml.etree.ElementTree
import seaborn as sns
import pylab as plt
from matplotlib.backends.backend_pdf import PdfPages
import urllib.request
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.style as style

%matplotlib inline

sns.set_style('whitegrid')
plt.rcParams['figure.figsize']=(5,4)

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [62]:
df = pd.read_csv('../../Task/BigTask/Parsed-results_S6.csv')

df.head(2)





display(df.head(2))

,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,party_ln,ideo,elections,teaparty,position_in_task,S6_rel,justification,url,off_page_count,ip_address,time
0,unit_0,REP_LIE_doc6,LIE,AFBS8D6KBKKKY,2,3,4,3,2,2,4,1,3,1,1,"After implementation of the law, 98% of Massac...",https://en.wikipedia.org/wiki/Massachusetts_he...,0,65.35.157.98,105.615
1,unit_0,DEM_FALSE_doc1,FALSE,AFBS8D6KBKKKY,2,3,4,3,2,2,4,1,3,2,1,She has literally cited him in the past.,https://twitchy.com/2014/11/13/jonathan-who-na...,3,65.35.157.98,133.090


In [6]:
#df["state"] = ""

#df['state'] = df.apply(
#    lambda row: urllib.request.urlopen("http://api.db-ip.com/v2/free/{}/stateProv".format(row.ip_address)).read(),
#    axis = 1)

#display(df.head())

In [63]:
df = df[['unit_id', 'ip_address']]
df = df.drop_duplicates().reset_index()
display(df.head())
df_ip = pd.DataFrame(columns=['unit_id', 'ip_info'])

for x in range(df.index.shape[0]):
    info = urllib.request.urlopen("http://api.db-ip.com/v2/free/{}/stateProvCode".format(df['ip_address'][x])).read()
    df_ip.loc[len(df_ip)] =  np.array([df['unit_id'][x], info])
print("done.")
display(df_ip.head())

,index,unit_id,ip_address
0,0,unit_0,65.35.157.98
1,8,unit_1,192.3.161.207
2,16,unit_2,107.198.59.190
3,24,unit_3,73.129.121.170
4,32,unit_4,184.183.28.233


done.


,unit_id,ip_info
0,unit_0,FL
1,unit_1,TX
2,unit_2,TX
3,unit_3,MD
4,unit_4,AZ


In [65]:
df_ip_grouped = df_ip.groupby(['ip_info']).count().reset_index()
df_ip_grouped.ip_info[0] = np.nan
df_ip_grouped.ip_info[36] = np.nan
display(df_ip_grouped)
df_ip_grouped = df_ip_grouped.dropna()

c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,ip_info,unit_id
0,NaN,16
1,AR,3
2,AZ,4
3,CA,15
4,CT,2
5,FL,13
6,GA,12
7,HI,4
8,IA,1
9,ID,1


In [66]:
def compute_relatives(table):
    tot = table['unit_id'].sum()
    display(tot)
    table['unit_id'] = table.apply(lambda row: round(row.unit_id / tot, 2), axis = 1)
    return table

df_ip_grouped = compute_relatives(df_ip_grouped)
display(df_ip_grouped)
df_ip_grouped.to_csv("../../Task/BigTask/S6_IP.csv", index=False, header=True)

175

,ip_info,unit_id
1,AR,0.02
2,AZ,0.02
3,CA,0.09
4,CT,0.01
5,FL,0.07
6,GA,0.07
7,HI,0.02
8,IA,0.01
9,ID,0.01
10,IL,0.06


In [7]:
df = pd.read_csv('../../Task/BigTask/S100_IP.csv')

In [9]:
def usa_label_map(table, label):
    scl = [
        [0.0, 'rgb(185,213,253)'],
        [0.2, 'rgb(160,210,231)'],
        [0.4, 'rgb(129,177,213)'],
        [0.6, 'rgb(61,96,167)'],
        [0.8, 'rgb(38,64,139)'],
        [1.0, 'rgb(15,8,75)']
    ]
    data = [go.Choropleth(
        colorscale = scl, autocolorscale = False, locations = table.ip_info,
        z = table[label].astype(float), locationmode = 'USA-states', text = '',
        marker = go.choropleth.Marker(
            line = go.choropleth.marker.Line(color = 'rgb(255,255,255)', width = 2 )),
        colorbar = go.choropleth.ColorBar(title = "Distribuzione")
    )]
    layout = go.Layout(
        title = go.layout.Title(text = 'Distribuzione Utenti USA | S100 Label'),
        geo = go.layout.Geo(
            scope = 'usa',
            projection = go.layout.geo.Projection(type = 'albers usa'), showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
    )
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig, filename = 'd3-cloropleth-map')
usa_label_map(df, 'unit_id')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~barbicchio/0 or inside your plot.ly account where it is named 'd3-cloropleth-map'
